In [1]:
#import libraries

import os
import json
import numpy
import datetime
import certifi
import pandas as pd
import urllib.parse

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
#set up connection to mysql and mongodb

mysql_args = {
    "uid" : "root",
    "pwd" : "Oakley2017good123!",
    "hostname" : "localhost",
    "dbname" : "classicmodels_dw"
}

mongodb_args = {
    "user_name" : "wmr6ku",
    "password" : "Oakley2017good123!",
    "cluster_name" : "Cluster0",
    "cluster_subnet" : "5nhij",
    "cluster_location" : "atlas",
    "db_name" : "classicmodels"
}

In [3]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [4]:
#create the classicmodels database to use

dst_dbname = "classicmodels_dw"

conn_str = f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

In [5]:
#populate mongodb with the data

#get the json file

client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(),'ds2002_midterm_project_data')

json_files = {"employees" : 'classicmodels_employees.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files) 

In [6]:
#extract data from employees collection

client = get_mongo_client(**mongodb_args)

query = {} 
collection = "employees"

df_employees = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_employees.head(2)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales


In [7]:
#get the csv file

data_dir = os.path.join(os.getcwd(),'ds2002_midterm_project_data')
data_file = os.path.join(data_dir, 'classicmodels_customers.csv')

df_customers = pd.read_csv(data_file, header=0)
df_customers.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800.0


In [8]:
#load payments and products data from sql as well

sql_products = "SELECT * FROM classicmodels.products;"
df_products = get_sql_dataframe(sql_products, **mysql_args)
df_products.head(2)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


In [9]:
sql_payments = "SELECT * FROM classicmodels.payments;"
df_payments = get_sql_dataframe(sql_payments, **mysql_args)
df_payments.head(2)

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,6066.78
1,103,JM555205,2003-06-05,14571.44


In [10]:
#create the dimension tables

sql_dim_date = "SELECT date_key, full_date FROM classicmodels_dw.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [11]:
#get the surrogate key

df_dim_payment_date = df_dim_date.rename(columns={"date_key" : "payment_date_key", "full_date" : "paymentDate"})
df_payments.paymentDate = df_payments.paymentDate.astype('datetime64[ns]').dt.date
df_payments = pd.merge(df_payments, df_dim_payment_date, on='paymentDate', how='left')
df_payments.drop(['paymentDate'], axis=1, inplace=True)
df_payments.head(2)

,customerNumber,checkNumber,amount,payment_date_key
0,103,HQ336336,6066.78,20041019
1,103,JM555205,14571.44,20030605


In [12]:
#create the primary keys

#primary key for employees

df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))
df_employees.head(2)

,employee_key,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,2,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales


In [13]:
#primary key for customers

df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))
df_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800.0


In [14]:
#primary key for products

df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head(2)

,product_key,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,2,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


In [15]:
#create the new tables

dataframe = df_employees
table_name = 'dim_employees'
primary_key = 'employee_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [16]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [17]:
dataframe = df_products
table_name = 'dim_products'
primary_key = 'product_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [18]:
#check that the dimension tables were created

sql_employees = "SELECT * FROM classicmodels_dw.dim_employees;"
df_dim_employees = get_sql_dataframe(sql_employees, **mysql_args)
df_dim_employees.head(2)

,employee_key,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,2,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales


In [19]:
sql_customers = "SELECT * FROM classicmodels_dw.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


In [20]:
sql_products = "SELECT * FROM classicmodels_dw.dim_products;"
df_products = get_sql_dataframe(sql_products, **mysql_args)
df_products.head(2)

,product_key,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,2,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


In [22]:
#create and populate the fact order tables

#join orders and order details

sql_orders = "SELECT * FROM classicmodels.orders;"
df_orders = get_sql_dataframe(sql_orders, **mysql_args)

sql_order_details = "SELECT * FROM classicmodels.orderdetails;"
df_order_details = get_sql_dataframe(sql_order_details, **mysql_args)

df_fact_orders = pd.merge(df_orders, df_order_details, on='orderNumber', how='right')
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_2248,50,55.09,2


In [23]:
#perform lookup operations

sql_dim_customers = "SELECT customer_key, customerNumber FROM classicmodels_dw.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_dim_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,customerNumber
0,1,103
1,2,112


In [24]:
sql_dim_products = "SELECT product_key, productCode FROM classicmodels_dw.dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **mysql_args)
df_dim_products.head(2)

,product_key,productCode
0,1,S10_1678
1,2,S10_1949


In [25]:
#merge fact orders and dim customers, make necessary transformations

df_fact_orders = pd.merge(df_fact_orders, df_dim_customers, on='customerNumber', how='inner')
df_fact_orders.drop(['customerNumber'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,customer_key
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,S18_1749,30,136.00,3,86
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,S18_2248,50,55.09,2,86


In [26]:
#merge fact orders and dim products, make necessary transformations

df_fact_orders = pd.merge(df_fact_orders, df_dim_products, on='productCode', how='inner')
df_fact_orders.drop(['productCode'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,30,136.0,3,86,23
1,10379,2005-02-10,2005-02-18,2005-02-11,Shipped,None,39,156.4,2,11,23


In [27]:
#get date keys

sql_dim_date = "SELECT date_key, full_date FROM classicmodels_dw.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [28]:
#lookup surrogate primary keys

#shipped date key

df_dim_shipped_date = df_dim_date.rename(columns={"date_key" : "shipped_date_key", "full_date" : "shippedDate"})
df_fact_orders.shippedDate = df_fact_orders.shippedDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_shipped_date, on='shippedDate', how='left')
df_fact_orders.drop(['shippedDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key,shipped_date_key
0,10100,2003-01-06,2003-01-13,Shipped,None,30,136.0,3,86,23,20030110.0
1,10379,2005-02-10,2005-02-18,Shipped,None,39,156.4,2,11,23,20050211.0


In [29]:
#order date key

df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "orderDate"})
df_fact_orders.orderDate = df_fact_orders.orderDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_order_date, on='orderDate', how='left')
df_fact_orders.drop(['orderDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,requiredDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key,shipped_date_key,order_date_key
0,10100,2003-01-13,Shipped,None,30,136.0,3,86,23,20030110.0,20030106
1,10379,2005-02-18,Shipped,None,39,156.4,2,11,23,20050211.0,20050210


In [30]:
#add fact order key

ordered_columns = ['orderNumber', 'product_key', 'customer_key', 'shipped_date_key', 'order_date_key',
                      'quantityOrdered', 'priceEach']
df_fact_orders = df_fact_orders[ordered_columns]

df_fact_orders.insert(0, "fact_order_key", range(1, df_fact_orders.shape[0]+1))
df_fact_orders.head(2)

,fact_order_key,orderNumber,product_key,customer_key,shipped_date_key,order_date_key,quantityOrdered,priceEach
0,1,10100,23,86,20030110.0,20030106,30,136.0
1,2,10379,23,11,20050211.0,20050210,39,156.4


In [31]:
#create the fact order table

dataframe = df_fact_orders
table_name = 'fact_orders'
primary_key = 'fact_order_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [32]:
#check the fact order table was created

sql_fact_orders = "SELECT * FROM classicmodels_dw.fact_orders;"
df_fact_orders = get_sql_dataframe(sql_fact_orders, **mysql_args)
df_fact_orders.head(2)

,fact_order_key,orderNumber,product_key,customer_key,shipped_date_key,order_date_key,quantityOrdered,priceEach
0,1,10100,23,86,20030110.0,20030106,30,136.0
1,2,10379,23,11,20050211.0,20050210,39,156.4


In [33]:
#author summarization query

sql_orders = """
    SELECT 
        p.productLine AS 'Product',
        c.customerName AS 'Customer',
        SUM(o.quantityOrdered) AS 'Total Quantity'
    FROM 
        classicmodels_dw.fact_orders AS o
    INNER JOIN 
        classicmodels_dw.dim_products AS p
        ON o.product_key = p.product_key
    INNER JOIN 
        classicmodels_dw.dim_customers AS c
        ON o.customer_key = c.customer_key
    GROUP BY 
        p.productLine, c.customerName;
"""

In [34]:
df_fact_orders = get_sql_dataframe(sql_orders, **mysql_args)
df_fact_orders

,Product,Customer,Total Quantity
0,Vintage Cars,Online Diecast Creations Co.,493.0
1,Vintage Cars,Euro+ Shopping Channel,1596.0
2,Vintage Cars,Rovelli Gifts,674.0
3,Vintage Cars,Motor Mint Distributors Inc.,44.0
4,Vintage Cars,"AV Stores, Co.",773.0
...,...,...,...
423,Vintage Cars,Boards & Toys Co.,31.0
424,Ships,Boards & Toys Co.,36.0
425,Classic Cars,Gift Ideas Corp.,29.0
426,Classic Cars,Daedalus Designs Imports,72.0
